# BIG QUERY

## IMPORTS

In [120]:
!pip install --upgrade google-cloud-language

Requirement already up-to-date: google-cloud-language in /usr/local/lib/python2.7/dist-packages
Requirement already up-to-date: google-gax<0.16dev,>=0.15.13 in /usr/local/lib/python2.7/dist-packages (from google-cloud-language)
Requirement already up-to-date: google-cloud-core<0.27dev,>=0.26.0 in /usr/local/lib/python2.7/dist-packages (from google-cloud-language)
Requirement already up-to-date: googleapis-common-protos[grpc]<2.0dev,>=1.5.2 in /usr/local/lib/python2.7/dist-packages (from google-cloud-language)
Requirement already up-to-date: enum34; python_version < "3.4" in /usr/local/lib/python2.7/dist-packages (from google-cloud-language)
Requirement already up-to-date: requests<3.0dev,>=2.13.0 in /usr/local/lib/python2.7/dist-packages (from google-gax<0.16dev,>=0.15.13->google-cloud-language)
Requirement already up-to-date: dill<0.3dev,>=0.2.5 in /usr/local/lib/python2.7/dist-packages (from google-gax<0.16dev,>=0.15.13->google-cloud-language)
Requirement already up-to-date: ply==3.8

In [123]:
import pandas as pd
from pandas.tools.plotting import autocorrelation_plot
from pandas.tools.plotting import scatter_matrix

import numpy as np

import matplotlib.pyplot as plt

import google.datalab.bigquery as bq

import tensorflow as tf

import random

# in order to print images in notebook
from IPython.display import Image
from IPython.core.display import HTML

## Get the data

We retrieve data from a Big Query table and put it in a panda frame.

In [124]:
# retrieve data from the Game of Thrones tweets in Big Query
# takes a long time be careful
got = bq.Query.from_table(bq.Table('Series.GOT'), fields=['Text','Language']).execute().result().to_dataframe() #.set_index('Date') to set an index from the rows

In [126]:
# we only keep english tweets and remove duplicates (to have less chance of encountering a "RT")
got_eng = got[got['Language']=="en"].drop_duplicates()
# remove these rows for good
got_eng = got_eng.dropna(axis=0, how='any')

In [127]:
# retrieve data from the Met Table in Big Query (public dataset)
# takes a long time be careful
met = bq.Query('SELECT a.object_id, b.description, b.score FROM `bigquery-public-data.the_met.vision_api_data` as a, unnest(labelAnnotations) as b').execute().result().to_dataframe() #.set_index('object_id') #to set an index from the rows

In [125]:
# quick description of the table
got.describe()

,Text,Language
count,1920830,1920830
unique,1016194,57
top,RT @Kreyz_: Rt si tu le vis bien. https://t.co...,en
freq,31573,1343756


In [129]:
# quick description of the table
got_eng.describe()

,Text,Language
count,724936,724936
unique,724936,1
top,Game of thrones has to be the best show on tv ...,en
freq,1,724936


In [130]:
got["Text"][1796586]

'Game Of Thrones fanatikleri buraya https://t.co/oxG5J5LLJC'

In [131]:
met.describe()

,object_id,score
count,935276.000000,935276.000000
mean,283209.129009,0.711613
std,187267.527939,0.119919
min,33.000000,0.500001
25%,103065.000000,0.619591
50%,261891.500000,0.696712
75%,421758.000000,0.792783
max,746939.000000,0.993372


In [132]:
met

,object_id,description,score
0,33644,weapon,0.843683
1,33644,product,0.780845
2,33644,dagger,0.519575
3,363331,sketch,0.757690
4,363331,drawing,0.719121
5,363331,relief,0.540961
6,14630,picture frame,0.737246
7,14630,rectangle,0.534322
8,14630,material,0.531604
9,263204,circle,0.678314


## Extract entities from a tweet via the Cloud Natural Language API

In [346]:
def extract_entities(text):
    # [START language_quickstart]
    # Imports the Google Cloud client library
    # [START migration_import]
    from google.cloud import language
    #from google.cloud.language import enums
    #from google.cloud.language import types
    # [END migration_import]

    # Instantiates a client
    # [START migration_client]
    client = language.Client()
    # [END migration_client]
    document = client.document_from_text(content=text)

    # analyze the feeling of the text
    sentiment = document.analyze_sentiment()
    print text
    print('Score: {}'.format(sentiment.sentiment.score))
    print('Magnitude: {}'.format(sentiment.sentiment.magnitude))
    
    
    # extract the entities from the text
    entities = document.analyze_entities().entities
    
    for ent in entities:
      print('Name: {}'.format(ent.name))
      print('Salience: {}'.format(ent.salience))
    
    return entities, sentiment

In [218]:
for i in got_eng.sample()["Text"]:
    print i, len(i)

Imagine staying with Game of Thrones this long just so you could tweet "um actually the show this bad" when they finally do a great episode 139


In [343]:
def extract_image(word):
    query = 'SELECT a.object_id, b.description, b.score FROM `bigquery-public-data.the_met.vision_api_data` as a, unnest(labelAnnotations) as b where REGEXP_CONTAINS(b.description, r\'^'+word+'$\') order by 3 desc'
    match = bq.Query(query).execute().result()#.to_dataframe() #.set_index('object_id') #to set an index from the rows
    if match.length>0:
        res = match.to_dataframe()
        print "Found "+str(len(res))+" results matching with word : " + word
    else:
        res = 1
        print "No matching found for word : " + word
    return res

In [377]:
for t in got_eng.sample()["Text"]:
    tweet = t
#tweet = "I love Sansa's dress in the last episode of Game of Thrones"
#tweet = "Earned 1 PSN trophy (1 silver) in Game of Thrones - https://t.co/QOY2DA7S8d"
entities, sentiment = extract_entities(tweet)

Earned 1 PSN trophy (1 silver) in Game of Thrones - https://t.co/QOY2DA7S8d
Score: 0.3
Magnitude: 0.3
Name: trophy
Salience: 0.5111663
Name: PSN
Salience: 0.3259483
Name: Game of Thrones
Salience: 0.16288543


## Match a tweet to an image

In [378]:
for ent in entities:
    res = extract_image(ent.name)
    # if there was a match for the word
    if type(res) != type(1):
        break
        
# if none of the words match
if type(res) == type(1):
    # negative sentiment
    if sentiment.sentiment.score < -0.6:
        res = extract_image("red")
    # positive sentiment
    elif sentiment.sentiment.score > 0.6:
        res = extract_image("green")
    else:
        res = extract_image("white")

Found 452 results matching with word : trophy


In [303]:
res

,object_id,description,score
0,322608,museum,0.815490
1,549495,museum,0.747989
2,466313,museum,0.712616
3,35928,museum,0.711160
4,549493,museum,0.702548
5,21926,museum,0.672080
6,49119,museum,0.639597
7,543937,museum,0.631614
8,590946,museum,0.611085
9,438024,museum,0.586542


## Display the associated image

In [379]:
query = 'SELECT original_image_url as url FROM `bigquery-public-data.the_met.images` where object_id='+str(res["object_id"][0])
image = bq.Query(query).execute().result().to_dataframe()
Image(url=image.get('url')[0])